In [ ]:
from spaco_py.SpaCoObject import SPACO
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from momentchi2 import wf, hbe

In [ ]:
sf = np.load(
    "/home/krastegar0/SpaCo_py/src/spaco_py/sf_mat.npy", allow_pickle=False
)
neighbor = np.load(
    "/home/krastegar0/SpaCo_py/src/spaco_py/A_mat.npy", allow_pickle=False
)

In [ ]:
#Loading the coorddinates 
coords = pd.read_excel('/home/krastegar0/SpaCo_py/src/spaco_py/coords_brain.xlsx')

In [ ]:
def plot_spatial_heatmap(coords, values, title="Spatial Heatmap", cmap="viridis", point_size=50):
    """
    Plots a discrete heatmap based on coordinates and corresponding values.
    Args:
        coords (numpy.ndarray): Array of shape (n_samples, 2) containing x and y coordinates.
        values (numpy.ndarray): Array of shape (n_samples,) containing values corresponding to the coordinates.
        title (str): Title of the plot.
        cmap (str): Colormap to use for the heatmap.
        point_size (int): Size of the points in the scatter plot.
    """
    # Ensure inputs are NumPy arrays
    coords = np.array(coords)
    values = np.array(values)
    # Create the scatter plot
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(coords[:, 0], coords[:, 1], c=values, cmap=cmap, s=point_size, edgecolor="k")
    plt.colorbar(scatter, label="Values")
    plt.title(title)
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.grid(True)
    plt.savefig(f'{title}.png', dpi=300, bbox_inches='tight')
    plt.show()
# Example usage
# Coordinates (x, y)
# coords = coords
# Values corresponding to the coordinates
# values = spac_patterns_X[:, 0]  # Use the first SpaCo pattern for demonstration
# Plot the heatmap
# plot_spatial_heatmap(coords, values, title="First SpaC",point_size=20)

In [ ]:
testObj = SPACO(neighbormatrix=neighbor, sample_features=sf)

In [ ]:
Pspac, Vkt =testObj.spaco_projection()

In [ ]:
non_random_eigvals = testObj._cache["non_random_eigvals"]
random_permuted_eigvals = testObj._cache["results_all"]


## Remarks on Fig 1
The distribution of eigenvalues shows the non-random eigenvalues in blue and the random permuted eigenvalues in orange.
The non-random eigenvalues represent the true structure of the data, while the random permuted eigenvalues serve as a null distribution for comparison.
We use the upper 95% confidence interval of the random permuted eigenvalues to determine the significance of the non-random eigenvalues.
The non-random eigenvalues that exceed this threshold are considered significant and indicate the presence of meaningful patterns in the data.
The plot provides a visual representation of the eigenvalue distributions, allowing us to assess the significance of the non-random eigenvalues in relation to the random permuted eigenvalues.
      
      

## Determing a metric to see if this pattern is actually spatially 
Here we see our first spatial pattern but how can we tell if the patterns that we see are actually spatial or they are just artifacts. We look at the length of the projections as the measure of "Smoothness" for a spatial gene 

In [ ]:
# need to generate the sigma matrix and sigma eigenvlaues for test statistic using 
testObj.spaco_test(Pspac[:, 0])

In [ ]:
coeff=testObj._cache["sigma_eigh"] # sum of these should be close to 1
coeff

In [ ]:
Q_sig=testObj._cache["sigma"]
Q_sig

### Going to use python momentchi2 package to run the weighted sum of chi2 distribution 
Here we attempt to use a weighted sum of chi2 package to determine if the length of the projected spaco pattern relates to a spatially relevant gene 

In [ ]:
graphLaplacian=testObj.graphLaplacian
graphLaplacian

In [ ]:
# calculating the sigma matrix and seeing if I made any mistakes there
sigma_tilda = graphLaplacian @ Vkt @ Vkt.T @ graphLaplacian
sigma=testObj._cache["sigma"]

In [ ]:
# recalculating test statistics and testing against the null distribution. 
graphLaplacian=testObj.graphLaplacian

# calculating the sigma matrix and seeing if I made any mistakes there
sigma_tilda = graphLaplacian @ Vkt @ Vkt.T @ graphLaplacian
sigma=testObj._cache["sigma"]

# Eigenvalues of the sigma matrix
sigma_eigenvalues = np.linalg.eigvals(sigma)
print(len(sigma_eigenvalues))
print(sum(sigma_eigenvalues)) # sum of these should be close to 1.

#### Achim Testing of SVG theory ####
1. Generate the null distribution from random chi-squared distribution variables and sigma eignenvalues from real data 
2. generate a random vector from a normal distribution and then calculate the test statistic of random vector against null distribution
3. compare test statistics and null distribution

In [ ]:
# generating the null statistic

chi = np.random.chisquare(df=1, size=len(sigma_eigenvalues))


In [ ]:
A_vec_null_distribution_Real = [coeff.T @ np.random.chisquare(df=1, size=len(sigma_eigenvalues)) for _ in range(10000)]
A_vec_null_distribution_Real
plt.hist(A_vec_null_distribution_Real, bins=30, color='orange', alpha=0.7, edgecolor='black')
plt.title("Histogram of Random Vector")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.show()

In [ ]:
#---Parameters----#
size = Vkt.shape[0]  # Length of the vector
mean = 0    # Mean of the normal distribution
std_dev = 1 # Standard deviation of the normal distribution

# Generate random vector
random_vector = np.random.normal(mean, std_dev, size) # random vector of normal expression values 

# Calculate the inner product
non_spatial_testStats = [
    np.random.normal(mean, std_dev, size).T @ graphLaplacian @ Vkt @ Vkt.T @ graphLaplacian @ np.random.normal(mean, std_dev, size)
    for _ in range(10000)
]

In [ ]:
# Plot the histogram
plt.hist(non_spatial_testStats, bins=30, color='blue', alpha=0.7, edgecolor='black', label="Random Vector Test Statistic")
plt.hist(A_vec_null_distribution_Real, bins=30, color='orange', alpha=0.7, edgecolor='black', label="Null Distribution from Real Lambdas")
plt.title("Histogram of Random Vector")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.legend(loc='upper right')
plt.show()

In [ ]:
Vkt.sum(axis=0) # Vk is constructed incorrectly (or something happens to it )

In [ ]:
#V_k^T * L *x  --> y  and then y^T * y # the issue was 
randomTestStats = []
for _ in range(10000):
    y = Vkt.T @ graphLaplacian @ np.random.normal(mean, std_dev, size)
    randomTestStats.append(y.T @ y)

#### Note: 
##### The issue was with generating the random test statistics. For some reason the calculation
##### was not correct. The random test statistics were not being generated correctly. Meaning in the code the test statistics were not calculated correctly as well

In [ ]:
plt.hist(randomTestStats, bins=30, color='blue', alpha=0.7, edgecolor='black', label="Random Vector Test Statistic")
plt.hist(A_vec_null_distribution_Real, bins=30, color='orange', alpha=0.7, edgecolor='black', label="Null Distribution from Real Lambdas")
plt.title("Histogram of Random Vector")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.legend(loc='upper right')
plt.show()